## Minimum working example for multi-worker dataset loading bug (duplicate filename)

In [ ]:
import yaml
import os
import webdataset as wds

from typing import List, Optional, Dict
import torch

from detectron2.data import detection_utils
from detectron2.structures import Boxes, BoxMode, Instances

from atek.dataset.omni3d_adapter import create_omni3d_webdataset
from atek.dataset.atek_webdataset import create_atek_webdataset, create_wds_dataloader

In [ ]:
def get_tars(tar_yaml, use_relative_path=False):
    with open(tar_yaml, "r") as f:
        tar_files = yaml.safe_load(f)["tars"]
    if use_relative_path:
        data_dir = os.path.dirname(tar_yaml)
        tar_files = [os.path.join(data_dir, x) for x in tar_files]
    return tar_files

train_list_file = "/data/home/louy/datadir/ase_wds/2024_02_19_200_sequences/train_tars.yaml"
tars = get_tars(train_list_file, use_relative_path=True)
ID_MAP_JSON="/data/home/louy/workdir/ATEK/data/ase_id_mapping.json"

omni3d_wds = create_omni3d_webdataset(tars, batch_size=32, category_id_remapping_json=ID_MAP_JSON, repeat = True)
dataloader = create_wds_dataloader(omni3d_wds, num_workers=8, pin_memory=False)
data_iter = iter(dataloader)

In [ ]:
for i in range(100):
    orig_data = next(data_iter)
    data = [x for x in orig_data if x["instances"].get("gt_classes").numel() > 0]
    seq_name = data[0]["sequence_name"]
    print(f"--- --- debug: processing data from sequence {seq_name}") 